<a href="https://colab.research.google.com/github/sirius70/NLP_HW2/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re

In [2]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [3]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.

def createBasicFeatures(corpus):
    # Extract texts and genres
    texts = [entry["text"] for entry in corpus]
    genres = [entry["genre"] for entry in corpus]

    # Create vectorizer that uses all tokens containing alphabetic characters
    vectorizer = CountVectorizer(
        token_pattern=r'[a-zA-Z]+'  # This pattern matches tokens with at least one alphabetic character
    )

    # Transform texts to feature matrix
    X = vectorizer.fit_transform(texts)

    # Get vocabulary
    vocab = vectorizer.get_feature_names_out().tolist()

    return X, genres, vocab

In [10]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

"""
    Creates bigram features for classification.

    Features:
      - Only bigrams (sequences of two consecutive words).
      - Captures short phrases that may signal genre differences.
        For example:
            - Comedy might have bigrams like "funny joke", "haha laugh".
            - Tragedy might have bigrams like "death scene", "dark fate".

    Returns:
      - X: sparse matrix of bigram features
      - y: list of correct genres (labels)
      - vocab: list of bigram features in the vocabulary
  """

def createInterestingFeatures(corpus):
  texts = [entry["text"] for entry in corpus]
  genres = [entry["genre"] for entry in corpus]

  vectorizer = CountVectorizer(
      ngram_range=(2, 2),     # only bigrams
      token_pattern=r'[a-zA-Z]+'  # keep tokens with alphabetic chars
  )

  X = vectorizer.fit_transform(texts)
  vocab = vectorizer.get_feature_names_out()

  return X, genres, vocab


In [11]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [7]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [8]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['you', 'helena', 'prospero', 'duke', 'sir', 'i', 'leontes', 'a', 'of', 'presently', 'preservers', 'preserver', 'preserved', 'pretty', 'prettiness', 'prettily', 'prettiest', 'prettier', 'presentment', 'preserving']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'o', 'brutus', 'lear', 'what', 'ham', 'and', 'rom', 'presentation', 'prettily', 'prettiest', 'prettier', 'pretia', 'pretext', 'pretense', 'pretending', 'presenters']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', 'ham', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [12]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['not a', 'and i', 'me and', 'which is', 'the duke', 'if you', 'she is', 'you have', 'to me', 'my love', 'the king', 'as you', 'i am', 'sir toby', 'of syracuse', 'quirk sir', 'quintus if', 'quintus martius', 'quit thee', 'quit their']
The most informative terms for neg are: ['the gods', 'art thou', 'the moor', 'give me', 'lady macbeth', 'my lord', 'to the', 'in his', 'know not', 'exeunt scene', 'caesar s', 'quirks and', 'quintus martius', 'quit thee', 'quit their', 'quit them', 'quit this', 'quit thy', 'quit with', 'quit you']
----------L2 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['the duke', 'sir toby', 'i am', 'if you', 'and i', 'princess of', 'i pray', 'of france', 'sir i', 'she is', 'of syracuse', 'a good', 'in love', 'of a', 'to her', 'i will', 'the king', 'my love', 'antipholus of', 'you will']
The most informative t

**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?

**Interpretation of Classifier Features**

With the basic count features, comedies are associated with words tied to lighter, social, and romantic settings. Character names like Helena, Prospero, Leontes, Hermia, and Lysander appear often, along with conversational words such as you, sir, your, me. These suggest the more personal and witty interactions typical of comedies. In contrast, tragedies highlight darker themes and characters: Iago, Othello, Brutus, Lear, Cassio, Desdemona, Caesar, Romeo. These names anchor the tragic plays in betrayal, political conflict, and death. The presence of words like him, his, thy also reflects the more formal, weighty tone of tragedies.

The interesting bigram features give even richer insights. In comedies, phrases like “the duke,” “sir toby,” “my love,” “in love,” “if you,” and “I pray” point to themes of love, mistaken identity, noble households, and humor. Specific references such as “sir toby” (from Twelfth Night) or “of syracuse” (from The Comedy of Errors) highlight comedic characters and settings. By contrast, tragedies emphasize expressions of power, death, and fate. Bigrams like “lady macbeth,” “my lord,” “the gods,” “of rome,” and “the moor” strongly evoke Shakespeare’s tragic figures and darker subject matter. Even stage directions like “exeunt scene” appear more often in tragedies, reflecting their heavier dramatic structure.

Overall, the classifiers suggest that comedies are marked by language of love, nobility, and playful dialogue, while tragedies are defined by death, politics, betrayal, and fate. These linguistic patterns align closely with how we understand Shakespeare’s genres, showing that even a simple bag-of-words or bigram model can uncover meaningful thematic distinctions.